In [1]:
# -*- coding: utf8 -*-
import urllib
from urllib.request import urlopen
import requests
import json
from bs4 import BeautifulSoup
from pprint import pprint
import random
from requests import Session
import time
import csv
import random
import http.cookiejar as cookielib

In [23]:
food_type = '肉制品'
check_flag = '不合格'
if check_flag == '合格':
    order_by = 1
else:
    order_by = 0
    
info = {}
info['food_type'] = food_type.encode('UTF-8')
info['check_flag'] = check_flag.encode('UTF-8')
info = urllib.parse.urlencode(info)
info = info.replace('&', '%22%2C%22').replace('=','%22%3A%22')+'%22%2C%22'    


total_info = []
session = requests.Session()
session.cookies = cookielib.LWPCookieJar(filename='sacspider')
try:
    session.cookies.load(ignore_discard=True)
except:
    print('cookie 文件未能加载')

    
url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
    
payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(0)

response = session.post(url=url, data=payload, headers=headers)
total_food = response.json()['resultData']['total']
        
print(food_type+'-'+check_flag+'共有{}个食品结果'.format(total_food))
food_page = total_food // 20
print('开始抓取数据.....')

for i in range(0,food_page,1):
#     print(i)
    url = 'http://www.foods12331.cn/food/detail/findFoodByPage.json'
    headers = {
        'Accept':'application/json',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
#         'Cookie':'CA0AE10AB2A6BFCF4D518AB76F3F0DBD',
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.foods12331.cn',
        'Origin':'http://www.foods12331.cn',
        'proxy-connection': "keep-alive",
        'cache-control': "no-cache",
        'Referer':'http://www.foods12331.cn/web/index.jsp?food_type=%E9%A5%AE%E6%96%99',
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
#     payload = "filters=%7B%22food_type%22%3A%22%E9%A5%AE%E6%96%99%22%2C%22check_flag%22%3A%22%E5%90%88%E6%A0%BC%22%2C%22order_by%22%3A%221%22%2C%22pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D".format(i)
    

    payload = 'filters=%7B%22'+info+'order_by%22%3A%22{}%22%2C%22'.format(order_by)+'pageNo%22%3A{}%2C%22pageSize%22%3A20%2C%22bar_code%22%3A%22%22%2C%22sampling_province%22%3A%22%22%2C%22name_first_letter%22%3Anull%2C%22food_name%22%3Anull%7D'.format(i)

    response = session.post(url=url, data=payload, headers=headers)
    if response.status_code != 200:
        print('[INFO ERROR] 第'+str(i)+'页内容无法爬取')
        continue
    for ele in response.json()['resultData']['items']:
        params = {}
        food_id = ele['id']
        food_name = ele['food_name']
        production_name = ele['production_name']
        check_num = ele['check_num']
        qualified_num = ele['qualified_num']
        unqualified_num = ele['unqualified_num']
        params['food_name'] = ele['food_name'].encode('UTF-8')
        params['production_name'] = ele['production_name'].encode('UTF-8')
        params['food_model'] = ele['food_model'].encode('UTF-8')
        payload = urllib.parse.urlencode(params)
        url = 'http://www.foods12331.cn/food/detail/getResult.json'
        headers = {
            'Accept':'*/*',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-CN,zh;q=0.8',
            'Connection':'keep-alive',
#             'Cookie':'JSESSIONID=473CB06EED2E2487C12EC5571F2EA61F',
            'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
            'Host':'www.foods12331.cn',
            'Origin':'http://www.foods12331.cn',
            'proxy-connection': "keep-alive",
            'cache-control': "no-cache",
            'Referer':'http://www.foods12331.cn/web/list.jsp?foodId={}&food_type=%E9%A5%AE%E6%96%99&food_condition='.format(food_id),
            'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'X-Requested-With':'XMLHttpRequest'
        }
        sleep_time = random.randint(2,4)
        time.sleep(sleep_time)
        response = session.post(url=url, data=payload, headers=headers)
        if response.status_code != 200:
            print('[INFO ERROR] food_id为'+food_id+'的食品无法抓取')
            continue
        for ele in response.json()['resultData']['foods']:
            food_dict = {}
            production_address = ele['production_adress']
            food_product_time = ele['food_product_time']
            food_model = ele['food_model']
            check_flag = ele['check_flag']
            sampling_name = ele['sampling_name']
            sampling_province = ele['sampling_province']
            check_projiect = ele['check_projiect']

            food_dict = {
                'food_name':food_name,
                'production_name':production_name,
                'check_num':check_num,
                'qualified_num':qualified_num,
                'unqualified_num':unqualified_num,
                'production_address':production_address,
                'food_product_time':food_product_time,
                'food_model':food_model,
                'check_flag':check_flag,
                'sampling_name':sampling_name,
                'sampling_province':sampling_province,
                'check_projiect':check_projiect
                }
            total_info.append(food_dict)
#             print(food_dict)
#         sleep_time = random.randint(1, 4)
#         time.sleep(sleep_time)
    session.cookies.save()
    print(u'已完成'+ str((i+1)*20) +u'个食品的数据抓取')
    
headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']
with open('肉制品-不合格.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)

肉制品-不合格共有1085个食品结果
开始抓取数据.....
已完成20个食品的数据抓取
已完成40个食品的数据抓取
已完成60个食品的数据抓取
已完成80个食品的数据抓取
已完成100个食品的数据抓取
已完成120个食品的数据抓取
已完成140个食品的数据抓取
已完成160个食品的数据抓取
已完成180个食品的数据抓取
已完成200个食品的数据抓取
已完成220个食品的数据抓取
已完成240个食品的数据抓取
已完成260个食品的数据抓取
已完成280个食品的数据抓取
已完成300个食品的数据抓取
已完成320个食品的数据抓取
已完成340个食品的数据抓取
已完成360个食品的数据抓取
已完成380个食品的数据抓取
已完成400个食品的数据抓取
已完成420个食品的数据抓取
已完成440个食品的数据抓取
已完成460个食品的数据抓取
已完成480个食品的数据抓取
已完成500个食品的数据抓取
已完成520个食品的数据抓取
已完成540个食品的数据抓取
已完成560个食品的数据抓取
已完成580个食品的数据抓取
已完成600个食品的数据抓取
已完成620个食品的数据抓取
已完成640个食品的数据抓取
已完成660个食品的数据抓取
已完成680个食品的数据抓取
已完成700个食品的数据抓取
已完成720个食品的数据抓取
已完成740个食品的数据抓取
已完成760个食品的数据抓取
已完成780个食品的数据抓取
已完成800个食品的数据抓取
已完成820个食品的数据抓取
已完成840个食品的数据抓取
已完成860个食品的数据抓取
已完成880个食品的数据抓取
已完成900个食品的数据抓取
已完成920个食品的数据抓取
已完成940个食品的数据抓取
[INFO ERROR] food_id为db52cc4d-dd0a-479e-87f3-43c50e8fede9的食品无法抓取
已完成960个食品的数据抓取
已完成980个食品的数据抓取
已完成1000个食品的数据抓取
已完成1020个食品的数据抓取
已完成1040个食品的数据抓取
已完成1060个食品的数据抓取
[INFO ERROR] food_id为1700277e-1944-4fd2-8ad4-2d74e3861f0f的食品无法抓取
已完成1080个食品的数据抓取


In [15]:
# headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']
# with open('sac_info_full.csv', 'w') as f:
#     f_csv = csv.DictWriter(f, headers)
#     f_csv.writeheader()
#     f_csv.writerows(total_info)
    
headers = ['food_name', 'production_name', 'check_num', 'qualified_num', 'unqualified_num', 'production_address', 'food_product_time', 'food_model', 'check_flag', 'sampling_name', 'sampling_province', 'check_projiect']
with open('肉制品-不合格.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)

In [10]:
with open('sac_info.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(total_info)

In [109]:
for i in response.json()['resultData']['items']:
    print(i['id'])

75b62200-7bc9-49a2-84e9-b2413e954691
713108da-05a4-4d50-810b-c238b7d73868
5d842f17-d9d5-4c7a-95ec-a4846d212724
a49014ab-fa02-4b0d-992a-08df00a70368
4e669154-b553-4c4b-9fe4-1046f5dd6a08
2671d4bd-a953-4629-8f69-105c17a18c4c
d5f47f7d-d8ff-43df-afa5-2f0f8aa6eb8b
a24bccf9-d3c5-4245-9e76-4fd849fdae79
ea42c5e9-86f9-4455-bfb1-e1530a93be7a
a0ed0413-49aa-4d8e-a04f-d2610ede5390
0e77456b-d433-42d9-a0df-fab4557c11fa
7943b332-6794-4faa-aa6c-66b0b2ee92c6
38700248-494c-4a45-92ca-a2c93424e344
57dbb88c-534a-49d7-8836-317ce5e4e6b3
2d26f6ba-3e4c-49cc-ae78-d888dfa8c3fc
641db086-0219-4dfb-90e1-b43d4d681cbc
02cda347-bac3-4115-87f0-56f98e475496
84ea0b4c-0160-4b7e-b80e-81d072adde89
00220ad6-1692-49b2-b5dc-f34b98888975
e70484d6-cc59-40c7-a804-6df4d32f5dfe


In [80]:
url = 'http://www.foods12331.cn/food/detail/getResult.json'
headers = {
    'Accept':'*/*',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.8',
    'Connection':'keep-alive',
    'Cookie':'JSESSIONID=473CB06EED2E2487C12EC5571F2EA61F',
    'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
    'Host':'www.foods12331.cn',
    'Origin':'http://www.foods12331.cn',
    'proxy-connection': "keep-alive",
    'cache-control': "no-cache",
    'Referer':'http://www.foods12331.cn/web/list.jsp?foodId=3f5b7d6b-ebb9-4cb2-824c-7be5df190982&food_type=%E9%A5%AE%E6%96%99&food_condition=',
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest'
}
s = requests.session()
payload = 'food_name=%E6%B3%89%E9%98%B3%E6%B3%89%E5%A4%A9%E7%84%B6%E7%9F%BF%E6%B3%89%E6%B0%B4&production_name=%E5%90%89%E6%9E%97%E6%A3%AE%E5%B7%A5%E9%9B%86%E5%9B%A2%E6%B3%89%E9%98%B3%E6%B3%89%E9%A5%AE%E5%93%81%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8&food_model=600mL%2F%E7%93%B6'
response = s.post(url=url, data=payload, headers=headers)

In [95]:
for ele in response.json()['resultData']['foods']:
    food_dict = {}
    production_address = ele['production_adress']
    food_product_time = ele['food_product_time']
    food_model = ele['food_model']
    check_flag = ele['check_flag']
    sampling_name = ele['sampling_name']
    sampling_province = ele['sampling_province']
    check_projiect = ele['check_projiect']

    food_dict = {
        'production_address':production_address,
        'food_product_time':food_product_time,
        'food_model':food_model,
        'check_flag':check_flag,
        'sampling_name':sampling_name,
        'sampling_province':sampling_province,
        'check_projiect':check_projiect
        }
    print(food_dict)
    print()

{'production_address': '吉林省抚松县泉阳镇新华街', 'sampling_name': '吉林森工集团泉阳泉饮品有限公司', 'sampling_province': '吉林', 'check_projiect': None, 'food_model': '600mL/瓶', 'food_product_time': '2016-10-12', 'check_flag': '合格'}

{'production_address': '吉林省抚松县泉阳镇新华街', 'sampling_name': '通化市东昌区福旺家超市', 'sampling_province': '吉林', 'check_projiect': None, 'food_model': '600mL/瓶', 'food_product_time': '2016-09-29', 'check_flag': '合格'}

{'production_address': '吉林省抚松县泉阳镇新华街', 'sampling_name': '天猫泉阳泉旗舰店', 'sampling_province': '浙江', 'check_projiect': '包括铅、总砷、镉、汞、亚硝酸盐、硝酸盐、色度、浑浊度、界限指标-锂、界限指标-锶、界限指标-锌、界限指标-碘化物、界限指标-偏硅酸、界限指标-硒、界限指标-游离二氧化碳、界限指标-溶解性总固体、硒、锑、铜、钡、铬、锰、镍、银、溴酸盐、硼酸盐、氟化物、耗氧量、挥发酚（以苯酚计）、氰化物、阴离子合成洗涤剂、大肠菌群、粪链球菌、铜绿假单胞菌、产气荚膜梭菌。', 'food_model': '600mL/瓶', 'food_product_time': '2016-9-19', 'check_flag': '合格'}

{'production_address': '吉林省抚松县泉阳镇新华街', 'sampling_name': '九台市城西幸福之佳仓储超市', 'sampling_province': '吉林', 'check_projiect': None, 'food_model': '600mL/瓶', 'food_product_time': '2016-09-18', 'check_flag': '合格'}

{'production